### [像有道词典这样的 JS 混淆加密应该怎么破](https://zhuanlan.zhihu.com/p/72219532)

In [26]:
import requests

url = 'http://fanyi.youdao.com/translate_o?smartresult=dict&smartresult=rule'
headers = {
    'Cookie' : 'OUTFOX_SEARCH_USER_ID=-1373274012@10.108.160.18; JSESSIONID=aaaPlhlJ1Cw-uhQGY1dZw; OUTFOX_SEARCH_USER_ID_NCOO=1180803013.2969046; ___rl__test__cookies=1566653920983',
    'Referer' : 'http://fanyi.youdao.com/',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}

from_data = {
    'i': '想你',
    'from': 'AUTO',
    'to': 'AUTO',
    'smartresult': 'dict',
    'client': 'fanyideskweb',
    'salt': '15666539209911',
    'sign': '4990c195323edb110727d4cd1234cbda',
    'ts': '1566653920991',
    'bv': '1fb71235800459b4adf0a96cef9082ce',
    'doctype': 'json',
    'version': '2.1',
    'keyfrom': 'fanyi.web',
    'action': 'FY_BY_CLICKBUTTION',
}

response = requests.post(url,data=from_data,headers=headers)
print(response.content)

b'{"errorCode":50}'


ts 在 js 中是这样的：

`r = "" + (new Date).getTime()`

那么在 Python 中就是这样的：

In [7]:
import time 

def get_ts():
    # 获取时间戳
    ts = int(time.time()*1000)
    return ts
    
print(get_ts())

1566656228878


bv 在 js 中是这样的：

`n.md5(navigator.appVersion)`

那么它在 Python 中就是这样的：

In [5]:
import hashlib

def get_bv():
    # 生成 bv
    appVersion = "5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
    m = hashlib.md5()
    m.update(appVersion.encode('utf-8'))
    bv = m.hexdigest()
    return bv

print(get_bv())

1fb71235800459b4adf0a96cef9082ce


salt 在 js 中是这样的：

`r + parseInt(10 * Math.random(), 10)`

那么它在 Python 中就是这样的：

In [9]:
import random

def get_salt(ts):
    # 生成 salt
    num = int(random.random()*10)
    salt = str(ts) + str(num)
    return salt

print(get_salt(get_ts()))

15666562896752


sign 在 js 中是这样的：

`n.md5("fanyideskweb" + e + i + "@6f#X3=cCuncYssPsuRUE")`

那么它在 Python 中就是这样的：

In [11]:
def get_sign(salt):
    # 生成 sign
    a = "fanyideskweb"
    b = '想你'
    c = salt
    d = "@6f#X3=cCuncYssPsuRUE"
    str_data = a+b+str(c)+d
    
    # md5 加密
    m = hashlib.md5()
    m.update(str_data.encode('utf-8'))
    sign = m.hexdigest()
    return sign

print(get_sign(get_salt(get_ts())))

c6fce514492057658401fc7daf40e644


所以我们获取 form data 的方法就是这样的：

In [30]:
import time,random,hashlib,requests,json

def get_ts():
    # 获取时间戳
    ts = int(time.time()*1000)
    return ts

def get_bv():
    # 生成 bv
    appVersion = "5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
    m = hashlib.md5()
    m.update(appVersion.encode('utf-8'))
    bv = m.hexdigest()
    return bv

def get_salt(ts):
    # 生成 salt
    num = int(random.random()*10)
    salt = str(ts) + str(num)
    return salt

def get_sign(salt):
    # 生成 sign
    a = "fanyideskweb"
    b = '想你'
    c = salt
    d = "n%A-rKaT5fb[Gy?;N5@Tj"
    str_data = a+b+c+d
    
    # md5 加密
    m = hashlib.md5()
    m.update(str_data.encode('utf-8'))
    sign = m.hexdigest()
    return sign

#print(get_sign(get_salt(get_ts())))

def get_from_data():
    ts = get_ts()
    salt = get_salt(ts)   
    from_data = {
        'i': '想你',
        'from': 'AUTO',
        'to': 'AUTO',
        'smartresult': 'dict',
        'client': 'fanyideskweb',
        'salt': salt,
        'sign': get_sign(salt),
        'ts': str(ts),
        'bv': get_bv(),
        'doctype': 'json',
        'version': '2.1',
        'keyfrom': 'fanyi.web',
        'action': 'FY_BY_CLICKBUTTION',
    }
    return from_data


url = 'http://fanyi.youdao.com/translate_o?smartresult=dict&smartresult=rule'
headers = {
    'Cookie' : 'OUTFOX_SEARCH_USER_ID=-1373274012@10.108.160.18; JSESSIONID=aaaPlhlJ1Cw-uhQGY1dZw; OUTFOX_SEARCH_USER_ID_NCOO=1180803013.2969046; ___rl__test__cookies=1566653920983',
    'Referer' : 'http://fanyi.youdao.com/',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}
print(get_from_data())
response = requests.post(url,data=get_from_data(),headers=headers)
dict_result = json.loads(response.content)
print('翻译结果：',dict_result['translateResult'][0][0]['tgt'])

{'i': '想你', 'from': 'AUTO', 'to': 'AUTO', 'smartresult': 'dict', 'client': 'fanyideskweb', 'salt': '15666585971630', 'sign': 'f9f2e941364934967fc7f7097dd73538', 'ts': '1566658597163', 'bv': '1fb71235800459b4adf0a96cef9082ce', 'doctype': 'json', 'version': '2.1', 'keyfrom': 'fanyi.web', 'action': 'FY_BY_CLICKBUTTION'}
翻译结果： Miss you.


### [JavaScript加密逻辑分析与Python模拟执行实现数据爬取](https://cuiqingcai.com/5024.html)

In [39]:
import requests
import execjs
import json

# Init environment
node = execjs.get()

# Params
method = 'GETDETAIL'
city = '深圳'
time_type = 'HOUR'
start_time = '2019-08-25 00:00:00'
end_time = '2019-08-25 23:00:00'

# Compile javascript
file = 'encryption.js'
ctx = node.compile(open(file).read())

# Get params
js = 'getEncryptedData("{0}", "{1}", "{2}", "{3}", "{4}")'.format(method, city, time_type, start_time, end_time)
params = ctx.eval(js)

# Get encrypted response text
api = 'https://www.aqistudy.cn/apinew/aqistudyapi.php'
response = requests.post(api, data={'d': params})

# Decode data
js = 'decodeData("{0}")'.format(response.text)
decrypted_data = ctx.eval(js)
data = json.loads(decrypted_data)
print(data)

{'success': True, 'errcode': 0, 'errmsg': 'success', 'result': {'success': True, 'data': {'total': 24, 'rows': [{'time': '2019-08-25 00:00:00', 'aqi': '64', 'pm2_5': '46', 'pm10': '72', 'co': '0.8', 'no2': '43', 'o3': '54', 'so2': '7', 'rank': '243'}, {'time': '2019-08-25 01:00:00', 'aqi': '48', 'pm2_5': '27', 'pm10': '48', 'co': '0.7', 'no2': '28', 'o3': '47', 'so2': '6', 'rank': '152'}, {'time': '2019-08-25 02:00:00', 'aqi': '22', 'pm2_5': '12', 'pm10': '22', 'co': '0.5', 'no2': '12', 'o3': '47', 'so2': '3', 'rank': '41'}, {'time': '2019-08-25 03:00:00', 'aqi': '20', 'pm2_5': '12', 'pm10': '20', 'co': '0.5', 'no2': '14', 'o3': '41', 'so2': '3', 'rank': '35'}, {'time': '2019-08-25 04:00:00', 'aqi': '22', 'pm2_5': '14', 'pm10': '22', 'co': '0.5', 'no2': '20', 'o3': '32', 'so2': '4', 'rank': '46'}, {'time': '2019-08-25 05:00:00', 'aqi': '26', 'pm2_5': '17', 'pm10': '26', 'co': '0.6', 'no2': '24', 'o3': '30', 'so2': '4', 'rank': '62'}, {'time': '2019-08-25 06:00:00', 'aqi': '27', 'pm2_5'